In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import datetime,os

import torch
import pandas as pd

import pickle

from fuzzywuzzy import fuzz

import re
from itertools import product

In [2]:
newdatapath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\'

In [46]:
basedatafiles = os.listdir(newdatapath+'01_SingleTransactions')

In [47]:
df = pd.DataFrame()

for file in basedatafiles:
    if file.lower().endswith('.xlsx'):
        tf = pd.read_excel(newdatapath+'01_SingleTransactions\\'+file)
    elif file.lower().endswith('.csv'):
        tf =pd.read_csv(newdatapath+'01_SingleTransactions\\'+file,sep=';',encoding='latin-1')
    else:
        continue
    tf['Data Source'] = file
    df = pd.concat([df,tf])

In [48]:
df = df.reset_index(drop=True)

In [53]:
df['Betrag'] = df['Betrag'].apply(lambda x: float(str(x).replace(',','.')))

In [58]:
rf = df.drop(columns=['Data Source']).drop_duplicates().reset_index(drop=True)

In [62]:
rf[
    (rf['Kontonummer/IBAN']=='DE07300308800013011001')&
    (rf['Buchungstag']=='24.09.19')
]

,Auftragskonto,Buchungstag,Valutadatum,Buchungstext,Verwendungszweck,Glaeubiger ID,Mandatsreferenz,Kundenreferenz (End-to-End),Sammlerreferenz,Lastschrift Ursprungsbetrag,Auslagenersatz Ruecklastschrift,Beguenstigter/Zahlungspflichtiger,Kontonummer/IBAN,BIC (SWIFT-Code),Betrag,Waehrung,Info,Kontonummer,BLZ
11,DE98773501100020557849,24.09.19,24.09.19,FOLGELASTSCHRIFT,D01-3392138-5522268 Prime Video 1QKOPREAQMXIB7UY,DE96ZZZ00000594888,oc37jPPdyn(18.Jnwk(4ZmqelPv?vb,1QKOPREAQMXIB7UY,NaN,NaN,NaN,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-4.99,EUR,Umsatz gebucht,NaN,NaN
12,DE98773501100020557849,24.09.19,24.09.19,FOLGELASTSCHRIFT,D01-7230805-3544654 Prime Video 4FF0418LUT7MU6QB,DE96ZZZ00000594888,oc37jPPdyn(18.Jnwk(4ZmqelPv?vb,4FF0418LUT7MU6QB,NaN,NaN,NaN,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-21.99,EUR,Umsatz gebucht,NaN,NaN
14,DE98773501100020557849,24.09.19,24.09.19,FOLGELASTSCHRIFT,D01-4453061-3787858 Prime Video 32RE7SHQPXC28NLW,DE96ZZZ00000594888,oc37jPPdyn(18.Jnwk(4ZmqelPv?vb,32RE7SHQPXC28NLW,NaN,NaN,NaN,AMAZON INSTANT VIDEO GERMANY GMBH,DE07300308800013011001,TUBDDEDDXXX,-3.99,EUR,Umsatz gebucht,NaN,NaN


In [63]:
col_rename_dict = {
    'Valutadatum':'Date_ordered', 
    'Buchungstag':'Date_booked', 
    'Buchungstext':'Text_transaction', 
    'Verwendungszweck':'Use',
    'Beguenstigter/Zahlungspflichtiger':'Contact', 
    'Kontonummer/IBAN':'AccNum',
    'Kontonummer':'AccNum',
    'Betrag':'Value_transaction',
    'Auftragskonto':'OrderAccount', 
    'Glaeubiger ID':'LenderID', 
    'Mandatsreferenz':'MandateReference',
    'Kundenreferenz (End-to-End)':'CustomerReference', 
    'Sammlerreferenz':'CollectorReference',
    'Lastschrift Ursprungsbetrag':'Amount0', 
    'Auslagenersatz Ruecklastschrift':'Amount1',
    'BIC (SWIFT-Code)':'BIC', 
    'BLZ':'BIC',
    'Waehrung':'Currency', 
    'Info':'Information'
}

In [67]:
rf = rf.rename(columns=col_rename_dict)

In [86]:
rf['Date_ordered'] = pd.to_datetime(rf['Date_ordered'])
rf['Date_booked'] = pd.to_datetime(rf['Date_booked'])

In [94]:
rf['Year_booked'] = rf['Date_booked'].apply(lambda x: x.year)
rf['Month_booked'] = rf['Date_booked'].apply(lambda x: x.month)
rf['Day_booked'] = rf['Date_booked'].apply(lambda x: x.day)

In [95]:
rf['Year_ordered'] = rf['Date_ordered'].apply(lambda x: x.year)
rf['Month_ordered'] = rf['Date_ordered'].apply(lambda x: x.month)
rf['Day_ordered'] = rf['Date_ordered'].apply(lambda x: x.day)

In [101]:
rf[['Year_booked','Month_booked','Value_transaction']].groupby(['Year_booked','Month_booked']).sum()

Value_transaction
Year_booked Month_booked                   
2019        1                         87.67
            2                        271.15
            3                      -2141.06
            4                      -3328.56
            5                        -50.70
            6                       3361.46
            7                       2108.58
            8                       1943.02
            9                        673.68
            10                      2217.74
            11                      1335.95
            12                      3650.02
2020        1                       1450.32
            2                       1008.66
            3                       -236.27
            4                        -26.29
            5                       1034.37
            6                       1685.28
            7                        162.17
            8                       -843.93
            9                        -81.00
            10                       -81.44
            11                     -4031.33
            12                     -1526.98

In [102]:
rf[rf['Date_ordered']>'2020-12-01']

,OrderAccount,Date_booked,Date_ordered,Text_transaction,Use,LenderID,MandateReference,CustomerReference,CollectorReference,Amount0,...,Currency,Information,AccNum,BIC,Year_booked,Month_booked,Day_booked,Year_ordered,Month_ordered,Day_ordered
169,DE98773501100020557849,2020-12-03,2020-12-03,KARTENZAHLUNG,SVWZ+2020-03-10T12.47Debitk.5 2021-12ABWA+TRAT...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,DE66763500000034003712,BYLADEM1ERH,2020,12,3,2020,12,3
170,DE98773501100020557849,2020-12-03,2020-12-03,ERSTLASTSCHRIFT,EREF+210+ZKLE 10.03.20 602/770100-S FMREF+MU03...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,DE14700500000002034343,BYLADEMM,2020,12,3,2020,12,3
171,DE98773501100020557849,2020-12-03,2020-12-03,ERSTLASTSCHRIFT,EREF+210+ZKLE 10.03.20 602/770100-S EMREF+MU03...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,DE14700500000002034343,BYLADEMM,2020,12,3,2020,12,3
203,DE98773501100020557849,2020-12-02,2020-12-02,GUTSCHR. UEBERWEISUNG,EREF+1798216 1SVWZ+0005051985/1798216 1/7.2.20...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,DE93700500000000055382,BYLADEMMXXX,2020,12,2,2020,12,2
204,DE98773501100020557849,2020-11-02,2020-12-02,ONLINE-UEBERWEISUNG,SVWZ+Kaution Wohnung Geschwister-Scholl-Strass...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,DE76120300001061152284,BYLADEM1001,2020,11,2,2020,12,2
603,DE98773501100020557849,2020-12-03,2020-12-03,KARTENZAHLUNG,2020-03-10T12:47 Debitk.5 2021-12,NaN,NaN,55548698004767100320124709,NaN,NaN,...,EUR,Umsatz gebucht,NaN,NaN,2020,12,3,2020,12,3
604,DE98773501100020557849,2020-12-03,2020-12-03,ERSTLASTSCHRIFT,"HUK-COBURG-Allgemeine, KRAFTFAHRT-VERSICHERUNG...",DE17ZZZ00000031285,MU038321695,210+ZKLE 10.03.20 602/770100-S F,NaN,NaN,...,EUR,Umsatz gebucht,NaN,NaN,2020,12,3,2020,12,3
605,DE98773501100020557849,2020-12-03,2020-12-03,ERSTLASTSCHRIFT,"HUK-COBURG-Allgemeine, KRAFTFAHRT-VERSICHERUNG...",DE17ZZZ00000031285,MU038321695,210+ZKLE 10.03.20 602/770100-S E,NaN,NaN,...,EUR,Umsatz gebucht,NaN,NaN,2020,12,3,2020,12,3
637,DE98773501100020557849,2020-12-02,2020-12-02,GUTSCHR. UEBERWEISUNG,0005051985/1798216 1/7.2.2020/Reise von Nuernb...,NaN,NaN,1798216 1,NaN,NaN,...,EUR,Umsatz gebucht,NaN,NaN,2020,12,2,2020,12,2
638,DE98773501100020557849,2020-11-02,2020-12-02,ONLINE-UEBERWEISUNG,Kaution Wohnung Geschwister-Scholl-Strasse Erl...,NaN,NaN,NaN,NaN,NaN,...,EUR,Umsatz gebucht,NaN,NaN,2020,11,2,2020,12,2


In [4]:
df_list = []

c_path = newdatapath+'01_SingleTransactions\\'
for c_file in os.listdir(c_path):
    f_name = c_file.split('.')[0].lower()
    f_term = c_file.split('.')[-1].lower()
    if f_term=='xls' or f_term=='xlsx':
        df_list.append(pd.read_excel(c_path+c_file).drop(columns=[
            'Glaeubiger ID',
            'Mandatsreferenz',
            'Kundenreferenz (End-to-End)',
            'Sammlerreferenz',
            'Lastschrift Ursprungsbetrag',
            'Auslagenersatz Ruecklastschrift'
        ],errors='ignore').rename(columns=col_rename_dict))
        if c_file=='20191215.xlsx':
            df_list[-1]['Value_transaction'] = df_list[-1]['Value_transaction'].apply(lambda x: float(str(x)[:-2]+'.'+str(x)[-2:]))
    elif f_term=='csv':
        df_list.append(pd.read_csv(c_path+c_file,sep=';',encoding='latin-1').drop(columns=[
            'Glaeubiger ID',
            'Mandatsreferenz',
            'Kundenreferenz (End-to-End)',
            'Sammlerreferenz',
            'Lastschrift Ursprungsbetrag',
            'Auslagenersatz Ruecklastschrift'
        ],errors='ignore').rename(columns=col_rename_dict))
        if c_file=='20191215.xlsx':
            df_list[-1]['Value_transaction'] = df_list[-1]['Value_transaction'].apply(lambda x: float(str(x)[:-2]+'.'+str(x)[-2:]))
    else:
        #print('Skipping file {}'.format(c_file))
        continue
    #print('Loaded file {}'.format(c_file))

In [5]:
df = pd.concat(df_list).reset_index(drop=True)

In [6]:
df['Use'] = df['Use'].apply(lambda x: 'Miete fuer Haesschen,du bist toll .)' if 'miete fuer haesschen' in x.lower() else x)
df['Use'] = df['Use'].apply(lambda x: 'lohn/gehalt' if 'lohn/gehalt' in x.lower() else x)

In [7]:
df['Date_booked'] = pd.to_datetime(df['Date_booked'],format='%d.%m.%y')
df['Date_ordered'] = pd.to_datetime(df['Date_ordered'],format='%d.%m.%y')

In [8]:
df['Value_transaction'] = df['Value_transaction'].apply(lambda x: str(x).replace(',','.')).astype(np.float64)

In [9]:
for colname in ['Text_transaction','Use','Contact']:
    df[colname] = df[colname].map(lambda x: re.sub(r'\W+', ' ', str(x)))

In [10]:
df = df.drop_duplicates(subset=[  
    'OrderAccount', 
    'Date_booked', 
    'Date_ordered', 
    'Text_transaction',
    'Use', 
    #'Contact', 
    'AccNum', 
    'BIC', 
    'Value_transaction', 
    'Currency',
    'Information'
],keep='first').reset_index(drop=True)

In [11]:
df['SpendingType'] = np.nan

# Identify cash withdrawls
df.loc[df['Text_transaction'].isin(['BARGELDAUSZAHLUNG']),'SpendingType'] = 'CASHOUT'

# Rent
df.loc[(df['Use'].apply(lambda x: 'miete' in x.lower()))&(df['Value_transaction']<=0),'SpendingType'] = 'RENT'

# Utilities
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'estw erlangerstadtwerkeag' in x.lower())),'SpendingType'] = 'UTILITIES POWER'
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'm net tele' in x.lower())),'SpendingType'] = 'UTILITIES INET'

# Phone bill
df.loc[(df['Use'].apply(lambda x: 'mobilfunk' in x.lower()))&(df['Contact'].apply(lambda x: 'telekom' in x.lower()))&(df['Value_transaction']<=0),'SpendingType'] = 'UTILITIES PHONEBILL'

# Identify salary
df.loc[(df['Text_transaction'].apply(lambda x: 'lohn gehalt' in x.lower())),'SpendingType'] = 'SALARY'

# Casino spending at SHS
df.loc[df['Use'].apply(lambda x: 'siemens casino card' in x.lower()),'SpendingType'] = 'FOOD&BEVERAGES'

# Union fees
df.loc[df['Contact']=='IGM Erlangen','SpendingType'] = 'MEMBERSHIP_FEES IGMETALL'

# ImmoScout fees
df.loc[df['Use'].apply(lambda x: 'immobilien' in x.lower()),'SpendingType'] = 'MEMBERSHIP_FEES IMMOSCOUT'

# Gym fees
df.loc[df['Contact']=='SLF Sportland Franken GmbH Co KG','SpendingType'] = 'MEMBERSHIP_FEES SPORTLAND'
df.loc[df['Use'].apply(lambda x: 'kieser training' in x.lower()),'SpendingType'] = 'MEMBERSHIP_FEES KIESER'

# Bonus payments from family
df.loc[df['Use'].isin([
    'MIETZUSCHUs F STUDENT F U B DEINHARD ', 
    'MIETE UND ANDERES ',
    'SVWZ TASCHENGELD', 
    'SVWZ MIETZUSCHUs F STUDENT F U B DEINHARD',
    'SVWZ MIETE UND ANDERES'
]),'SpendingType'] = 'BONUSIN_FAMILY'
df.loc[(df['SpendingType'].isna())&(df['Use']=='TASCHENGELD ')&(df['Contact']=='Hermann Kuespert'),'SpendingType'] = 'BONUSIN_FAMILY'

# Insurance fees
df.loc[(df['Contact'].apply(lambda x: 'hansemerkur' in x.lower()))|(df['Contact']=='Techniker Krankenkasse'),'SpendingType'] = 'INSURANCE TKK'
df.loc[(df['Contact'].apply(lambda x: 'huk coburg' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'INSURANCE HUK'
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'haftpflicht' in x.lower())),'SpendingType'] = 'INSURANCE HAFTPFL'
df.loc[(df['SpendingType'].isna())&(df['Contact']=='ARAG KRANKENVERSICHERUNG AG'),'SpendingType'] = 'INSURANCE ARAG' 
df.loc[(df['SpendingType'].isna())&(df['Contact']=='Siemens BKK'),'SpendingType'] = 'INSURANCE SBK'

# Credit card bill
df.loc[df['Contact']=='KREDITKARTENABRECHNUNG','SpendingType'] = 'CREDITCARD'

# Contact lenses
df.loc[(df['Use'].apply(lambda x: 'schaefer nohe' in x.lower()))|(df['Contact'].apply(lambda x: 'schaefer nohe' in x.lower())),'SpendingType'] = 'MEDICAL OPTICIAN'

# Gene sequencing 23andMe
df.loc[df['Use'].apply(lambda x: '23andme' in x.lower()),'SpendingType'] = 'MEDICAL MISC'

# Amazon videos
df.loc[df['Contact']=='AMAZON INSTANT VIDEO GERMANY GMBH','SpendingType'] = 'AMAZONVIDEOS'
df.loc[df['Use'].apply(lambda x: 'prime video' in x.lower()),'SpendingType'] = 'AMAZONVIDEOS'

# Amazon shopping
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'amzn mktp' in x.lower() or 'amazon' in x.lower())),'SpendingType'] = 'AMAZONSHOPPING'

# Entertainment
df.loc[df['Use'].apply(lambda x: 
        'spotify' in x.lower() or 
        'dancarlin' in x.lower() or
        'steam games' in x.lower()
),'SpendingType'] = 'ENTERTAINMENTMISC'

# Depot
df.loc[df['Text_transaction']=='WERTPAPIERE','SpendingType'] = 'STOCKDEPOT'

# Debit card bill
df.loc[(df['Use'].apply(lambda x: 'debitk' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'DEBITCARD PAYMENT'
df.loc[(df['Text_transaction'].apply(lambda x: 'sepa' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'DEBITCARD TRANSFER'

# Lent money backflow
df.loc[(df['SpendingType'].isna())&(df['Text_transaction'].apply(lambda x: 'gutschr' in x.lower())),'SpendingType'] = 'LENTBACK_PRIVATE'

# Rent deposit
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'kaution' in x.lower())),'SpendingType'] = 'DEPOSIT'

# Miscellaneous PayPal expenses
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'paypal'in x.lower())),'SpendingType'] = 'PAYPAL_MISC'

# Costs for the account
df.loc[(df['SpendingType'].isna())&(df['AccNum'].apply(lambda x: x=='0000000000' or str(x)=='nan'))&(df['Use'].apply(lambda x: 'entgeltabrechnung' in x.lower())),'SpendingType'] = 'BANKACCOUNTCOST'
df.loc[(df['SpendingType'].isna())&(df['AccNum']=='0020557849')&(df['BIC']=='77350110')&(df['Text_transaction']=='ABSCHLUSS'),'SpendingType'] = 'BANKACCOUNTCOST'

# Label known miscellanii
known_misc = pickle.load(open(newdatapath+'05_METADATA\\known_misc.pkl','rb'))
df.loc[(df['SpendingType'].isna())&(df[['Text_transaction','Use','Contact','AccNum','BIC']].apply(lambda x: '|||'.join([str(x) for x in list(x)]) in known_misc,axis=1)),'SpendingType'] = 'KNOWNMISC'

# Identify unknown miscellanii
df.loc[df['SpendingType'].isna(),'SpendingType'] = '__UNKNOWN__' 

In [12]:
# Major expense groupings
dict_spending_type2group = {
    'CASHOUT':'CASHOUT', 
    'BANKACCOUNTCOST':'BANKACCOUNTCOST', 
    'RENT':'RENT', 
    'MEMBERSHIP_FEES SPORTLAND':'MEMBERSHIP_FEES',
    'FOOD&BEVERAGES':'UTILITIES', 
    'UTILITIES PHONEBILL':'UTILITIES', 
    'UTILITIES POWER':'UTILITIES',
    'MEMBERSHIP_FEES IGMETALL':'MEMBERSHIP_FEES', 
    'SALARY':'SALARY', 
    'AMAZONVIDEOS':'AMAZON',
    'ENTERTAINMENTMISC':'MISC', 
    'CREDITCARD':'CREDITCARDBILL', 
    'MEDICAL OPTICIAN':'MEDICAL',
    'UTILITIES INET':'UTILITIES', 
    'MEMBERSHIP_FEES KIESER':'MEMBERSHIP_FEES', 
    'DEBITCARD PAYMENT':'CREDITCARDBILL',
    'PAYPAL_MISC':'MISC', 
    'DEBITCARD TRANSFER':'MISC',
    'LENTBACK_PRIVATE':'LENTBACK',
    'INSURANCE SBK':'INSURANCE', 
    'BONUSIN_FAMILY':'BONUS', 
    'STOCKDEPOT':'STOCKDEPOT', 
    'AMAZONSHOPPING':'AMAZON',
    'MEMBERSHIP_FEES IMMOSCOUT':'MEMBERSHIP_FEES', 
    'INSURANCE ARAG':'INSURANCE', 
    'KNOWNMISC':'KNOWNMISC',
    'INSURANCE HAFTPFL':'INSURANCE', 
    'INSURANCE TKK':'INSURANCE', 
    'MEDICAL MISC':'MEDICAL',
    'INSURANCE HUK':'INSURANCE', 
    'DEPOSIT':'DEPOSIT',
    '__UNKNOWN__':'__UNKNOWN__'
}
df['ExpenseGroup'] = df['SpendingType'].map(dict_spending_type2group)

In [13]:
df.to_excel(storepath+'STF.xlsx')

In [31]:
df

,OrderAccount,Date_booked,Date_ordered,Text_transaction,Use,Contact,AccNum,BIC,Value_transaction,Currency,Information,SpendingType,ExpenseGroup
0,DE98773501100020557849,2019-10-02,2019-10-01,BARGELDAUSZAHLUNG,2019 10 01T22 08 Debitk 5 2021 12,DECHSENDOR SPARKASSE ERLANGEN DE,DE63763500009000481424,BYLADEM1ERH,-50.00,EUR,Umsatz gebucht,CASHOUT,CASHOUT
1,DE98773501100020557849,2019-09-30,2019-10-01,ABSCHLUSS,Abrechnung 30 09 2019 siehe Anlage,nan,0020557849,77350110,0.00,EUR,Umsatz gebucht,BANKACCOUNTCOST,BANKACCOUNTCOST
2,DE98773501100020557849,2019-09-30,2019-10-01,ENTGELTABSCHLUSS,Entgeltabrechnung siehe Anlage,nan,0000000000,77350110,-4.50,EUR,Umsatz gebucht,BANKACCOUNTCOST,BANKACCOUNTCOST
3,DE98773501100020557849,2019-09-30,2019-09-30,DAUERAUFTRAG,Miete fuer Haesschen du bist toll,Elaine Fernandez,DE46763500000044116613,BYLADEM1ERH,-235.00,EUR,Umsatz gebucht,RENT,RENT
4,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,V Nr 47589 09 19 Beitrag 24 90,SLF Sportland Franken GmbH Co KG,DE88763500000000062725,BYLADEM1ERH,-24.90,EUR,Umsatz gebucht,MEMBERSHIP_FEES SPORTLAND,MEMBERSHIP_FEES
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,DE98773501100020557849,2019-11-22,2019-11-22,GUTSCHR UEBERWEISUNG,0005020864 1797970 1 19 11 2019 OTBI Reise nac...,Siemens AG,DE93700500000000055382,BYLADEMMXXX,791.11,EUR,Umsatz gebucht,LENTBACK_PRIVATE,LENTBACK
910,DE98773501100020557849,2019-11-22,2019-11-22,EIGENE KREDITKARTENABRECHN,VISA ABR 490762XXXXXX5575,KREDITKARTENABRECHNUNG,9902292060,77350110,-1480.02,EUR,Umsatz gebucht,CREDITCARD,CREDITCARDBILL
911,DE98773501100020557849,2019-11-21,2019-11-21,FOLGELASTSCHRIFT,D Nr M501545176 INV 201910394963 vom 14 11 201...,M net Telekommunikations GmbH,DE74700500000000028273,BYLADEMMXXX,-29.89,EUR,Umsatz gebucht,UTILITIES INET,UTILITIES
912,DE98773501100020557849,2019-11-20,2019-11-18,BARGELDAUSZAHLUNG,2019 11 18T17 57 Debitk 5 2021 12 Original 160...,ATM LHR T2 Arrs Ex 2 Middlesex GB 2,DE69500500000959566019,HELADEFF,-193.86,EUR,Umsatz gebucht,CASHOUT,CASHOUT


In [14]:
# Extract relevant information
tf = df[['Date_booked','Value_transaction']]
tf = tf.drop(index=tf[tf['Value_transaction']>=0].index).reset_index(drop=True)

In [15]:
# Generate interesting time indices
tf['YYYY-MM'] = pd.to_datetime(tf['Date_booked'].apply(lambda x: x.strftime('%Y-%m')))
tf['YYYY-MM-DD'] = pd.to_datetime(tf['Date_booked'].apply(lambda x: x.strftime('%Y-%m-%d')))
# Extract single date identifiers
#tf['DD'] = tf['Date_booked'].apply(lambda x: x.day)
#tf['MM'] = tf['Date_booked'].apply(lambda x: x.month)
#tf['YYYY'] = tf['Date_booked'].apply(lambda x: x.year)
# Drop unformatted column
tf = tf.drop(columns=['Date_booked'])

In [16]:
# Accumulate by months and days 
rf_monthly = tf[['YYYY-MM','Value_transaction']].groupby(['YYYY-MM']).sum().abs()
rf_daily = tf[['YYYY-MM-DD','Value_transaction']].groupby(['YYYY-MM-DD']).sum().abs()

In [17]:
# Extract full date
rf_monthly['Date'] = rf_monthly.apply(lambda x: x.index)
rf_daily['Date'] = rf_daily.apply(lambda x: x.index)
# Extract years
rf_daily['Year'] = rf_daily.apply(lambda x: x['Date'].year,axis=1)
rf_monthly['Year'] = rf_monthly.apply(lambda x: x['Date'].year,axis=1)
# Extract months
rf_daily['Month'] = rf_daily.apply(lambda x: x['Date'].month,axis=1)
rf_monthly['Month'] = rf_monthly.apply(lambda x: x['Date'].month,axis=1)
# Extract days
rf_daily['Day'] = rf_daily.apply(lambda x: x['Date'].day,axis=1)
rf_monthly['Day'] = rf_monthly.apply(lambda x: x['Date'].day,axis=1)

In [18]:
# Determine current dates of interest
c_date = tf['YYYY-MM-DD'].max()
p_date = c_date-pd.DateOffset(months=1)
pp_date = c_date-pd.DateOffset(months=2)
ppp_date = c_date-pd.DateOffset(months=3)

In [19]:
# Current month
start_date = '{}-{}-{}'.format(str(c_date.year),str(str(c_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(c_date.year if c_date.month!=12 else c_date.year+1),str(str((c_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==c_date.year)&(rf_daily['Month']==c_date.month)][['Value_transaction']]
m0 = tt.reindex(c_index,fill_value=0)

In [20]:
# Current month - 1
start_date = '{}-{}-{}'.format(str(p_date.year),str(str(p_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(p_date.year if p_date.month!=12 else p_date.year+1),str(str((p_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==p_date.year)&(rf_daily['Month']==p_date.month)][['Value_transaction']]
m1 = tt.reindex(c_index,fill_value=0)

In [21]:
# Current month - 2
start_date = '{}-{}-{}'.format(str(pp_date.year),str(str(pp_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(pp_date.year if pp_date.month!=12 else pp_date.year+1),str(str((pp_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==pp_date.year)&(rf_daily['Month']==pp_date.month)][['Value_transaction']]
m2 = tt.reindex(c_index,fill_value=0)

In [22]:
# Current month - 3
start_date = '{}-{}-{}'.format(str(ppp_date.year),str(str(ppp_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(ppp_date.year if ppp_date.month!=12 else ppp_date.year+1),str(str((ppp_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==ppp_date.year)&(rf_daily['Month']==ppp_date.month)][['Value_transaction']]
m3 = tt.reindex(c_index,fill_value=0)

In [23]:
writer = pd.ExcelWriter(storepath+'RunningMonthlyExp.xlsx',engine='openpyxl')

m0.to_excel(writer,sheet_name='p0')
m1.to_excel(writer,sheet_name='p1')
m2.to_excel(writer,sheet_name='p2')
m3.to_excel(writer,sheet_name='p3')

In [24]:
bf = df[['Date_booked','Value_transaction']]

pf = bf[bf['Value_transaction']>0].reset_index(drop=True)
nf = bf[bf['Value_transaction']<0].reset_index(drop=True)

pf['YYYY-MM'] = pd.to_datetime(pf['Date_booked'].apply(lambda x: x.strftime('%m-%Y')))
nf['YYYY-MM'] = pd.to_datetime(nf['Date_booked'].apply(lambda x: x.strftime('%m-%Y')))

In [25]:
pf = pf.drop(columns=['Date_booked'])
nf = nf.drop(columns=['Date_booked'])

In [26]:
pf = pf.groupby(['YYYY-MM']).sum()
nf = nf.groupby(['YYYY-MM']).sum()

In [27]:
mf = pd.merge(nf,pf,on='YYYY-MM').rename(columns={
    'Value_transaction_x':'expenses',
    'Value_transaction_y':'revenue'
})
mf['expenses'] = mf['expenses'].abs()

In [28]:
mf.to_excel(storepath+'MonthlyExpRevAggregate.xlsx')

In [29]:
at = pd.read_excel(newdatapath+'02_AccountTotal\\AccountTracker.xlsx',parse_dates=['Date'],index_col='Date')

In [30]:
at.to_excel(storepath+'ATR.xlsx')